## Finetune a Voice Cloning model with Tortoise-TTS on Amazon SageMaker
This notebook contains the reference code for training your own Text-To-Speech (TTS) model using tortoise-tts model as the base. 
Tortoise-tts is a popular open source voice cloning model that demonstrates great results in generating strong multi-voice with highly realistic prosody and intonation. For more information about tortoise-tts, please refer to this [link](https://arxiv.org/pdf/2305.07243)

In this notebook, we are going to train a custom voice model using Amazon SageMaker training job.

The original code and processes of finetuning tortoise-tts model is derived from this [github repo](https://github.com/JarodMica/ai-voice-cloning). 

Let's first install all the dependencies

In [ ]:
%pip install sagemaker boto3 -q

In [ ]:
import sagemaker
import boto3
from sagemaker.estimator import Estimator
from sagemaker.local import LocalSession # uncomment this if you are testing locally
from sagemaker import get_execution_role

Leverage an IAM that would allow SageMaker to access resources for the training job. 

In [ ]:
role = get_execution_role()
session = sagemaker.Session()

In [ ]:
sm_training_s3_input_location= "[S3 input location prefix]" # e.g. "s3://[your bucket]/data/visual-dubbing/training/input/"
sm_training_s3_model_location= "[S3 output location prefix]" # e.g. s3://[your bucket]/data/visual-dubbing/training/output/"

In [ ]:
hyperparams = {
                'yaml' : '/opt/ml/input/data/training/train.yaml' # the yaml file location. Keep the path as is, it'll resolve to the local directory in the container. 
              }

In [ ]:
estimator = Estimator(
    image_uri = "[the ECR image URI for tortoise TTS]" # "[aws-account].dkr.ecr.[region].amazonaws.com/tortoise-tts-training",
    role=role,
    instance_count=1,
    instance_type = "ml.g5.2xlarge",
    volume_size=225,
    output_path=sm_training_s3_model_location,
    sagemaker_session=session,
    hyperparameters=hyperparams,
)

In [ ]:
estimator.fit({'training': sm_training_s3_input_location})